# 

# Métrica: 4_2_01 — Coincidencia entre empresas con participación del declarante y el RUPC
| Caso          | Condición                                                                   | Resultado   |
| ------------- | --------------------------------------------------------------------------- | ----------- |
| **NO_APLICA** | El declarante **NO** reporta participación en empresas (lista vacía o nula) | "NO_APLICA" |
| **SIN_DATO**  | Tiene participación, pero falta nombre en DI **o** no es posible comparar   | "SIN_DATO"  |
| **NO_CUMPLE** | La empresa del declarante **sí** aparece en el RUPC                         | "NO_CUMPLE" |
| **CUMPLE**    | La empresa NO aparece en el RUPC                                            | "CUMPLE"    |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata
import openpyxl
import os

METRIC_ID = "4_2_XX_COINCIDENCIA_EMPRESA_RUPC"
RUPC_FILE = "rupc.xlsx"


# -------------------------------------------------------------
# AUXILIARES
# -------------------------------------------------------------
def extraer_valor(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None


def normalizar_texto(v):
    if not v:
        return None
    v = str(v).strip().upper()
    v = v.replace('"', '').replace("'", "")
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    while "  " in v:
        v = v.replace("  ", " ")
    return v


def limpiar_header(v):
    if not v:
        return ""
    v = str(v).strip().upper()
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    while "  " in v:
        v = v.replace("  ", " ")
    return v


# -------------------------------------------------------------
# CARGAR ARCHIVO EXCEL RUPC
# -------------------------------------------------------------
def cargar_empresas_rupc_desde_excel():
    empresas = set()

    if not os.path.exists(RUPC_FILE):
        print(f"❌ No se encontró el archivo {RUPC_FILE}")
        return empresas

    try:
        print(f"📄 Leyendo archivo Excel: {RUPC_FILE}")
        wb = openpyxl.load_workbook(RUPC_FILE, read_only=True, data_only=True)
        ws = wb.active

        header_raw = [c.value for c in next(ws.iter_rows(min_row=2, max_row=2))]
        header = [limpiar_header(c) for c in header_raw]

        idx_nombre = None
        for i, h in enumerate(header):
            if "NOMBRE" in h and "EMPRESA" in h:
                idx_nombre = i + 1
                break

        if idx_nombre is None:
            print("❌ No se encontró ninguna columna que coincida con 'Nombre de la empresa'.")
            print("Encabezados detectados:")
            for h in header:
                print(f" - {h}")
            return empresas

        print(f"✔ Columna de empresa detectada en índice {idx_nombre}")

        for row in ws.iter_rows(min_row=2):
            celda = row[idx_nombre - 1].value
            nombre = normalizar_texto(celda)
            if nombre:
                empresas.add(nombre)

    except Exception as e:
        print("❌ Error leyendo el archivo RUPC:", e)

    print(f"🔎 Empresas cargadas del RUPC: {len(empresas)}")
    return empresas


# -------------------------------------------------------------
# EVALUADOR DE LA MÉTRICA
# -------------------------------------------------------------
def evaluar_metrica(doc, empresas_rupc):
    nombre_di = normalizar_texto(
        extraer_valor(doc, "declaracion.intereses.participacion.participacion.nombreEmpresa")
    )

    if not nombre_di:
        return "NO_APLICA"

    if nombre_di is None or nombre_di == "":
        return "SIN_DATO"

    if nombre_di in empresas_rupc:
        return "NO_CUMPLE"

    return "CUMPLE"


# -------------------------------------------------------------
# PROCESAMIENTO MONGO
# -------------------------------------------------------------
def procesar_metrica_4_2_xx():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        empresas_rupc = cargar_empresas_rupc_desde_excel()

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]

        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.intereses.participacion": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc, empresas_rupc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"⚠️ Error en documento {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"> {total} documentos procesados...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print("Total procesados:", total)
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_2_xx()



🚀 Procesando métrica 4_2_XX_COINCIDENCIA_EMPRESA_RUPC...

📄 Leyendo archivo Excel: rupc.xlsx
✔ Columna de empresa detectada en índice 3
🔎 Empresas cargadas del RUPC: 24851
> 2000 documentos procesados...
> 4000 documentos procesados...
> 6000 documentos procesados...
> 8000 documentos procesados...
> 10000 documentos procesados...
> 12000 documentos procesados...
> 14000 documentos procesados...
> 16000 documentos procesados...
> 18000 documentos procesados...
> 20000 documentos procesados...
> 22000 documentos procesados...
> 24000 documentos procesados...
> 26000 documentos procesados...
> 28000 documentos procesados...
> 30000 documentos procesados...
> 32000 documentos procesados...
> 34000 documentos procesados...
> 36000 documentos procesados...
> 38000 documentos procesados...
> 40000 documentos procesados...
> 42000 documentos procesados...
> 44000 documentos procesados...
> 46000 documentos procesados...
> 48000 documentos procesados...
> 50000 documentos procesados...
> 5200